# ACGAN => Auxiliary Classifier gan 
### this network is similar to the CGAN but instead of passing one hot label vector to both generator and discriminator we just pass it to the generator and give the discriminator another classification task to preform we will make the model classify the output and this belived to make the network learn better so we used this architecture 

![ACGAN](images/ACGAN.png)

In [1]:
import tensorflow as tf 
import os 
from tensorflow.keras.layers import Dense , Conv2D ,concatenate, Conv2DTranspose , Flatten , Reshape  , BatchNormalization , Activation, Flatten ,LeakyReLU
from tensorflow.keras.models import Sequential 
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.datasets import mnist 
import numpy as np 
import matplotlib.pyplot as plt 
import math 
import os 
import argparse


# Generator 

In [2]:
class Generator(tf.keras.Model) : 
    def __init__(self) : 
        super().__init__()  
        self.dense_1 = Dense(7*7*128 ) 
        self.reshape = Reshape((7,7,128))

        
        self.sequential = Sequential([
             BatchNormalization() ,
             Activation('relu') , 
             Conv2DTranspose(128 , 5 , strides = 2 , padding = 'same') , 
             BatchNormalization() ,
             Activation('relu') ,
             Conv2DTranspose(64 , 5 , strides = 2  , padding = 'same') ,
             BatchNormalization() ,
             Activation('relu') ,
             Conv2DTranspose(32 , 5   , padding = 'same') ,
             BatchNormalization() ,
             Activation('relu') ,
             Conv2DTranspose(1 , 5  , padding = 'same') , 
             Activation('sigmoid')
        ])
        
    def call(self , inputs ) :
        X_vector , one_hot_vector = inputs
        X = concatenate([X_vector , one_hot_vector] , axis = 1) 
        X = self.reshape(self.dense_1(X ))
        return self.sequential(X)    
    
    

# Discriminator 

In [3]:
class Discriminator(tf.keras.Model) : 
    def __init__(self) : 
        super().__init__() 
        self.sequential = Sequential([
            LeakyReLU(.2), 
            Conv2D(32 , 5 , padding ='same' , strides = 2) , 
            LeakyReLU(.2),
            Conv2D(64 , 5 , padding ='same' , strides = 2) ,
            LeakyReLU(.2),
            Conv2D(128 , 5 , padding ='same' , strides = 2) ,
            LeakyReLU(.2),
            Conv2D(256 , 5 , padding ='same'  ) ,
            Flatten() , 
        ])
        self.output_1 = Dense(1 , activation ='sigmoid') 
        self.output_2 = Dense(10 , activation ='softmax' )
        

        
        
    def call(self, inputs ) : 
        X =  self.sequential(inputs) 
        output_1 = self.output_1(X) 
        output_2 = self.output_2(X) 
        return output_1 , output_2 
        
        

In [47]:
def train_step(images , labels , generator , discriminator  , loss_fn_1 , loss_fn_2 , gen_optimizer , dis_optimizer):
    noise = tf.random.normal([64, 100])
    fake_labels = np.eye(10)[np.random.choice(10,64)]

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        generated_images = generator((noise , fake_labels), training=True)

        real_output_1 ,  real_output_2 = discriminator((images  ) , training=True)
        fake_output_1 , fake_output_2 = discriminator((generated_images ) , training=True)

        gen_loss_1 = loss_fn_1(fake_output_1 , tf.ones_like(fake_output_1))
        gen_loss_2 = loss_fn_2(fake_output_2 , fake_labels )
        gen_loss_2 = tf.cast(gen_loss_2, tf.float32)
        gen_loss = gen_loss_1 +  gen_loss_2
        
        disc_real_loss_1 = loss_fn_1(real_output_1, tf.ones_like(real_output_1)) 
        disc_fake_loss_1 = loss_fn_1(fake_output_1 , tf.zeros_like(fake_output_1))
        disc_real_loss_2 = loss_fn_2(real_output_2, labels ) 
        disc_fake_loss_2 = loss_fn_2(fake_output_2 , fake_labels)
        disc_fake_loss_2 = tf.cast(disc_fake_loss_2 , tf.float32)
        disc_loss = (disc_real_loss_1 + disc_fake_loss_1) + (disc_real_loss_2 + disc_fake_loss_2)
        
    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    gen_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    dis_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))
    return gen_loss , disc_loss 

In [48]:
def plot_images(generator,
                noise_input,
                noise_class,
                show=False,
                step=0,
                model_name="gan"):
    os.makedirs(model_name, exist_ok=True)
    filename = os.path.join(model_name, "%05d.png" % step)
    images = generator.predict([noise_input, noise_class])
    print(model_name , " labels for generated images: ", np.argmax(noise_class, axis=1))
    plt.figure(figsize=(2.2, 2.2))
    num_images = images.shape[0]
    image_size = images.shape[1]
    rows = int(math.sqrt(noise_input.shape[0]))
    for i in range(num_images):
        plt.subplot(rows, rows, i + 1)
        image = np.reshape(images[i], [image_size, image_size])
        plt.imshow(image, cmap='gray')
        plt.axis('off')
    plt.savefig(filename)
    if show:
        plt.show()
    else:
        plt.close('all')

In [49]:
def build_and_train_models(epochs):
    # load MNIST dataset
    (x_train, y_train ), (_, _) = mnist.load_data()

    # reshape data for CNN as (28, 28, 1) and normalize
    x_train = np.reshape(x_train, [-1, 28, 28, 1])
    x_train = x_train.astype('float32') / 255
    y_train = tf.keras.utils.to_categorical(y_train , 10)

    ds = tf.data.Dataset.from_tensor_slices((x_train , y_train) ) 
    ds = ds.shuffle(6000).batch(64)
    ds = ds.take(500)

    noise_class = np.eye(10)[np.arange(0, 16) % 10]
    cross_entropy = tf.keras.losses.BinaryCrossentropy()
    categorical_loss = tf.keras.losses.CategoricalCrossentropy()
    generator_optimizer = tf.keras.optimizers.Adam(1e-4)
    discriminator_optimizer = tf.keras.optimizers.Adam(1e-4)

    # build discriminator model
    discriminator = Discriminator()
    # build generator model
    generator = Generator()
    
    for epoch in range(epochs) : 
        gen_losses = []
        dis_losses = []
        for batch_images , labels  in ds : 
            gen_loss , dis_loss =  train_step(images = batch_images , labels = labels , generator = generator  , discriminator = discriminator
            , loss_fn_1 =cross_entropy , loss_fn_2 = categorical_loss , gen_optimizer = generator_optimizer , dis_optimizer =discriminator_optimizer )
            gen_losses.append(gen_loss) 
            dis_losses.append(dis_loss)
        plot_images(generator ,tf.random.normal([16, 100]), noise_class , step = epoch  )
        print(f'gen_loss : {np.mean(gen_losses)} , dis_Loss : {np.mean(dis_losses)}')


In [ ]:
build_and_train_models(10)

2022-08-16 12:30:42.385383: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-08-16 12:30:42.524213: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 320ms/step
gan  labels for generated images:  [0 1 2 3 4 5 6 7 8 9 0 1 2 3 4 5]
gen_loss : 13.751045227050781 , dis_Loss : 27.290725708007812
1/1 [==============================] - 0s 11ms/step
gan  labels for generated images:  [0 1 2 3 4 5 6 7 8 9 0 1 2 3 4 5]
gen_loss : 1.6227505207061768 , dis_Loss : 20.044973373413086
1/1 [==============================] - 0s 8ms/step
gan  labels for generated images:  [0 1 2 3 4 5 6 7 8 9 0 1 2 3 4 5]
gen_loss : 1.6559436321258545 , dis_Loss : 19.80335235595703
1/1 [==============================] - 0s 8ms/step
gan  labels for generated images:  [0 1 2 3 4 5 6 7 8 9 0 1 2 3 4 5]
gen_loss : 0.22876526415348053 , dis_Loss : 17.400470733642578
1/1 [==============================] - 0s 8ms/step
gan  labels for generated images:  [0 1 2 3 4 5 6 7 8 9 0 1 2 3 4 5]
gen_loss : 0.007297718897461891 , dis_Loss : 16.738245010375977
1/1 [==============================] - 0s 8ms/step
gan  labels for generated images: 